In [3]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from threading import Thread
import pandas as pd
import time
import os
from datetime import datetime, timedelta

class IBApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.done = False

    def historicalData(self, reqId, bar):
        self.data.append([bar.date, bar.open, bar.high, bar.low, bar.close, bar.volume])

    def historicalDataEnd(self, reqId, start, end):
        self.done = True
        print("Download complete")

def create_contract(symbol):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    return contract


In [4]:
def fetch_data(symbol, date_str, filename):
    app = IBApp()
    app.connect("127.0.0.1", 7497, clientId=2)

    def run_loop():
        app.run()

    thread = Thread(target=run_loop, daemon=True)
    thread.start()
    time.sleep(1)

    contract = create_contract(symbol)
    app.reqHistoricalData(
        reqId=1,
        contract=contract,
        endDateTime=date_str,
        durationStr='1 D',
        barSizeSetting='1 min',
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1,
        keepUpToDate=False,
        chartOptions=[]
    )

    while not app.done:
        time.sleep(1)

    app.disconnect()

    df = pd.DataFrame(app.data, columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume'])
    os.makedirs('ibkr_1min_data_ibapi/', exist_ok=True)
    df.to_csv(f'ibkr_1min_data_ibapi/{filename}', index=False)
    print(f"{symbol} data saved.")

# Example: Fetch yesterday’s data
fetch_data('AAPL', (datetime.now() - timedelta(days=1)).strftime("%Y%m%d %H:%M:%S"), 'AAPL_1min.csv')


ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usopt.nj
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:apachmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefhk
ERROR 1 2174 Warning: You submitted request with date-time attributes without explicit time zone. Please switch to use yyyymmdd-hh:mm:ss in UTC or use instrument time zone, like US/Eastern. Implied time zone functionality will be removed in the next API release
ERROR 1 2176 Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 1849.09 to 1849


Download complete
AAPL data saved.
